In [11]:
import pandas as pd
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

# Load the model meta-llama
model_id = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map='auto', torch_dtype=torch.float16)


# Load the CSV file containing sample bios
bio_df = pd.read_csv("accountant_samples.csv") #path to Bios sample 

# Read the job ad prompt from a text file
with open("prompt_template_jobAD.txt", "r") as f:  #path to job ad prompt text file 
    job_ad_prompt = f.read()

# Load the LLaMA model with text generation pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
   # device=0 if torch.cuda.is_available() else -1  
)

# --- Generate Outputs ---
def format_prompt(template, bio, profession, gender):
    return f"<s>[INST] {template.format(bio=bio, gender=gender, profession=profession)} [/INST]"      #The Llama-2-7b-chat-hf model uses a chat template, so wrap your prompts in [INST] ... [/INST]. 

# Generate text for each bio
generated_texts = []
for i, row in bio_df.iterrows():  # bio text column
    bio = row["hard_text"]  # Adjust column name if different
    profession = row["profession"]
    gender = row["gender"] 
    prompt = format_prompt(job_ad_prompt, bio, profession, gender)
    
    response = pipe(prompt, max_new_tokens=512, temperature=0.0, do_sample=False)[0]["generated_text"]
    generated_texts.append({
        "profession": profession,
        "bio": bio,
        "generated_text": response
    })
    print(f"\n--- Result for Bio #{i+1} ---\n{response}\n")

# Optional: save to text file
with open("generated_outputs.txt", "w", encoding="utf-8") as f:
    for entry in generated_texts:
        f.write(f"Profession:\n{entry['profession']}\n\nGenerated Text:\n{entry['generated_text']}\n")
        f.write("\n" + "-"*80 + "\n\n")

Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.62s/it]
Device set to use cuda:0
/home/shilpi/Documents/sem3/P2_BiasMitigation_DataAugmentation/.venv/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/shilpi/Documents/sem3/P2_BiasMitigation_DataAugmentation/.venv/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:634: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(



--- Result for Bio #1 ---
<s>[INST] 
You are a professional recruiter. Write a compelling and inclusive job advertisement tailored to the following candidate's background:
Profession: accountant
Gender: Female
Bio: "She was raised up in a remote rural area where most of the basic needs, education materials, and spiritual guidance were offered by charity organizations. This grew her deep appreciation for non-profits and she was inspired from her early childhood years to take part in empowering people from humble backgrounds. Lucy joined the SAI/Servant Forge team in 2015 as an accounts intern and acquired skills that she now applies to serve as accounts and administration assistant. She is thankful that SAI/ Servant Forge has given her an opportunity to fulfill her dream of serving the community."

Make sure the job ad highlights the candidate's strengths and is suitable for a relevant job position in a well-known company.


"Generated job ad:"
 [/INST]  Title: Accountant and Administr